In [7]:
import json
import regex as re
import os

In [8]:
cwd = os.getcwd()
cwd

'/Users/justinchen/Desktop/GitHub/Polar-Bears-Synthetic-Data/YOLO'

In [12]:
def get_coords(file):
    final_list = []
    if file['annotations'] == []:
        return final_list
    for i in range(len(file['annotations'])):
        temp_list = []
        temp_list.append(0) #file['annotations'][i]['id']
        for j in range(len(file['annotations'][i]['bbox'])):
            temp_list.append(file['annotations'][i]['bbox'][j])
        final_list.append(temp_list)
    final_list.append(file['filename'])
    return final_list

In [13]:
def calc_new_coords(coord_list, img_size):
    new_coord_list = []
    temp_coord_list = coord_list[:-1]
    for box in temp_coord_list:
        new_box = [box[0]]
        x_min = box[1]
        y_min = box[2]
        w = box[3]
        h = box[4]
        
        x_max = x_min + w
        y_max = y_min + h
        
        x_cent = (x_min + x_max) / 2
        y_cent = (y_min + y_max) / 2
        print(x_cent)
        
        new_box.extend([x_cent / img_size, y_cent / img_size, w / img_size, h / img_size])
        new_coord_list.append(new_box)
    new_coord_list.append(coord_list[-1])
    return new_coord_list

In [14]:
def write_txt(yolo_list):
    if yolo_list == []:
        return 
    file_name = yolo_list[-1][:-4]
    #new directory to save txt file to
    save_path = '/Users/justinchen/Desktop/Synthetic-Data/YOLO/Datasets/0.25Dataset/obj'
    completeName = os.path.join(save_path, file_name)
    f = open(completeName + ".txt", "x")
    for i in range(len(yolo_list)-1):
        for j in range(len(yolo_list[i])):
            f.write(str(yolo_list[i][j]) + " ")
        f.write("\n")
    print('txt file written')
    f.close()

In [9]:
import os

In [ ]:
import os
# assign directory
directory = '/Users/justinchen/Desktop/GitHub/Polar-Bears-Synthetic-Data/50f8d5d7-0add-4323-a3fa-7fdcda8b9e81/annotations'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        JsonFile = open(f)
        file = json.load(JsonFile)
        coord_list = get_coords(file)
        if not coord_list == []:
            new_coord_list = calc_new_coords(coord_list, 320)
            print(new_coord_list)
            write_txt(new_coord_list)

In [16]:
directory = '/Users/justinchen/Desktop/Synthetic-Data/YOLO/Datasets/0.25Dataset/obj'
count = 0
for filename in os.listdir(directory):
    newfilename = filename.split('.')[0]
#     print(filename)
#     print(filename + ".txt")
    if not os.path.exists(directory + '/' + newfilename + ".txt"):
        print(filename)
        os.remove(directory + '/' + filename)
        count += 1
print("Images removed:", count)
print("Images left:", 1500-count)

Images removed: 0
Images left: 1500


In [ ]:
import os
import cv2
from xml.dom.minidom import parseString
from lxml.etree import Element, SubElement, tostring
import numpy as np
from os.path import join

## coco classes
YOLO_CLASSES = ('polar bear')
train = '/Users/justinchen/Desktop/Synthetic-Data/YOLO/Datasets/0.30Dataset/obj'
for filename in os.listdir(train):
    if '.txt' in filename:

In [ ]:
## converts the normalized positions  into integer positions
def unconvert(class_id, width, height, x, y, w, h):

    xmax = int((x*width) + (w * width)/2.0)
    xmin = int((x*width) - (w * width)/2.0)
    ymax = int((y*height) + (h * height)/2.0)
    ymin = int((y*height) - (h * height)/2.0)
    class_id = int(class_id)
    return (class_id, xmin, xmax, ymin, ymax)


## path root folder
ROOT = 'coco'


## converts coco into xml 
def xml_transform(root, classes):  
    class_path  = join(root, 'labels')
    ids = list()
    l=os.listdir(class_path)
    
    check = '.DS_Store' in l
    if check == True:
        l.remove('.DS_Store')
        
    ids=[x.split('.')[0] for x in l]   

    annopath = join(root, 'labels', '%s.txt')
    imgpath = join(root, 'images', '%s.jpg')
    
    os.makedirs(join(root, 'outputs'), exist_ok=True)
    outpath = join(root, 'outputs', '%s.xml')

    for i in range(len(ids)):
        img_id = ids[i] 
        if img_id == "classes":
            continue
        if os.path.exists(outpath % img_id):
            continue
        print(imgpath % img_id)
        img= cv2.imread(imgpath % img_id)
        height, width, channels = img.shape # pega tamanhos e canais das images

        node_root = Element('annotation')
        node_folder = SubElement(node_root, 'folder')
        node_folder.text = 'VOC2007'
        img_name = img_id + '.jpg'
    
        node_filename = SubElement(node_root, 'filename')
        node_filename.text = img_name
        
        node_source= SubElement(node_root, 'source')
        node_database = SubElement(node_source, 'database')
        node_database.text = 'Coco database'
        
        node_size = SubElement(node_root, 'size')
        node_width = SubElement(node_size, 'width')
        node_width.text = str(width)
    
        node_height = SubElement(node_size, 'height')
        node_height.text = str(height)

        node_depth = SubElement(node_size, 'depth')
        node_depth.text = str(channels)

        node_segmented = SubElement(node_root, 'segmented')
        node_segmented.text = '0'

        target = (annopath % img_id)
        if os.path.exists(target):
            label_norm= np.loadtxt(target).reshape(-1, 5)

            for i in range(len(label_norm)):
                labels_conv = label_norm[i]
                new_label = unconvert(labels_conv[0], width, height, labels_conv[1], labels_conv[2], labels_conv[3], labels_conv[4])
                node_object = SubElement(node_root, 'object')
                node_name = SubElement(node_object, 'name')
                node_name.text = classes[new_label[0]]
                
                node_pose = SubElement(node_object, 'pose')
                node_pose.text = 'Unspecified'
                
                
                node_truncated = SubElement(node_object, 'truncated')
                node_truncated.text = '0'
                node_difficult = SubElement(node_object, 'difficult')
                node_difficult.text = '0'
                node_bndbox = SubElement(node_object, 'bndbox')
                node_xmin = SubElement(node_bndbox, 'xmin')
                node_xmin.text = str(new_label[1])
                node_ymin = SubElement(node_bndbox, 'ymin')
                node_ymin.text = str(new_label[3])
                node_xmax = SubElement(node_bndbox, 'xmax')
                node_xmax.text =  str(new_label[2])
                node_ymax = SubElement(node_bndbox, 'ymax')
                node_ymax.text = str(new_label[4])
                xml = tostring(node_root, pretty_print=True)  
                dom = parseString(xml)
        print(xml)  
        f =  open(outpath % img_id, "wb")
        #f = open(os.path.join(outpath, img_id), "w")
        #os.remove(target)
        f.write(xml)
        f.close()     

In [ ]:
xml_transform(ROOT, YOLO_CLASSES)

In [14]:
def write_data():
    f = open("obj.data", "x")
    f.write('classes = 1')
    f.write("\n")
    
    f.write('train = data/train.txt')
    f.write("\n")
    
    f.write('valid = data/test.txt')
    f.write("\n")
    
    f.write('names = data/obj.names')
    f.write("\n")
    
    f.write('backup = /mydrive/yolov4/training')
    f.write("\n")
    
    print('.data file written')
    f.close()

In [15]:
write_data()

.data file written


In [95]:
def write_names():
    f = open("obj.names", "x")
    f.write('polar bear')
    f.close()

In [97]:
write_names()